# Result analysis


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json


In [2]:
with open('result/TIE_MarkupLM/stage_2/qas_eval_results_.json', "r") as f:
    qas_eval_results_json = json.load(f)

qas_eval_results = pd.DataFrame(qas_eval_results_json).transpose()   

In [3]:
qas_eval_results.head()

,exact,f1,pos
jo100000100000,1.0,1.0,1.0
jo100000100001,0.0,0.0,0.5
jo100000100002,1.0,1.0,1.0
jo100000100003,0.0,0.0,0.5
jo100000100004,0.0,0.0,0.5


In [4]:
with open('result/TIE_MarkupLM/stage_2/tag_predictions_.json', "r") as f:
    tag_predictions_json = json.load(f)

tag_predictions = pd.Series(tag_predictions_json, name='tag_predictions')



tag_predictions.index.name= 'id'
tag_predictions = tag_predictions.reset_index()
tag_predictions.head()

,id,tag_predictions
0,jo100000100000,10
1,jo100000100001,9
2,jo100000100002,10
3,jo100000100003,6
4,jo100000100004,6


In [12]:
with open('result/TIE_MarkupLM/stage_2/predictions_.json', "r") as f:
    predictions_json = json.load(f)
predictions = pd.Series(predictions_json, name='predictions')



predictions.index.name= 'id'
predictions = predictions.reset_index()
predictions.head()

,id,predictions
0,jo100000100000,Hamburg
1,jo100000100001,Unlimited Jobs GmbH
2,jo100000100002,Hamburg
3,jo100000100003,2. Koordinator*in / Sachbearbeiter*in zur Erst...
4,jo100000100004,2. Koordinator*in / Sachbearbeiter*in zur Erst...


In [13]:
with open('data/jobs/10/dataset.csv', "r") as f:
    dataset = pd.read_csv(f, sep=';')




In [14]:
dataset.head()

,question,id,element_id,answer_start,answer
0,In welchem Ort ist der Job Koordinator*in / Sa...,jo100000100000,10,0,Hamburg
1,Wo ist der Job Koordinator*in / Sachbearbeiter...,jo100000100001,10,0,Hamburg
2,Wo kann man Koordinator*in / Sachbearbeiter*i...,jo100000100002,10,0,Hamburg
3,Wo gibt es Arbeit als Koordinator*in / Sachbea...,jo100000100003,10,0,Hamburg
4,Wer sucht nach einer Koordinator*in / Sachbear...,jo100000100004,9,0,Unlimited Jobs GmbH


In [7]:
tag_predictions_dataset = pd.merge(tag_predictions, dataset, on='id')
tag_predictions_dataset.head()

,id,tag_predictions,question,element_id,answer_start,answer
0,jo100000100000,10,In welchem Ort ist der Job Koordinator*in / Sa...,10,0,Hamburg
1,jo100000100001,9,Wo ist der Job Koordinator*in / Sachbearbeiter...,10,0,Hamburg
2,jo100000100002,10,Wo kann man Koordinator*in / Sachbearbeiter*i...,10,0,Hamburg
3,jo100000100003,6,Wo gibt es Arbeit als Koordinator*in / Sachbea...,10,0,Hamburg
4,jo100000100004,6,Wer sucht nach einer Koordinator*in / Sachbear...,9,0,Unlimited Jobs GmbH


In [15]:


def location_match(df):
    tag_offset_false = 0
    counter = 0
    for index, row in df.iterrows():
        if int(row.element_id) != int(row.tag_predictions):
            tag_offset_false += abs(int(row.element_id) - int(row.tag_predictions))
            counter += 1
    return tag_offset_false / counter

def all_location_match(df):
    tag_offset_false = 0
    counter = df.shape[0]
    for index, row in df.iterrows():
        
        tag_offset_false += abs(int(row.element_id) - int(row.tag_predictions))
            
    return tag_offset_false / counter

In [16]:
whole_dataset_offset = location_match(tag_predictions_dataset)
whole_dataset_all_offset = all_location_match(tag_predictions_dataset)
print(whole_dataset_offset)
print(whole_dataset_all_offset)

2.111842105263158
1.1145833333333333


In [134]:
whole_dataset_offset

2.111842105263158

In [17]:
tag_predictions_dataset = pd.merge(predictions, tag_predictions_dataset, on='id')

In [18]:
tag_predictions_dataset.head(
)

,id,predictions,tag_predictions,question,element_id,answer_start,answer
0,jo100000100000,Hamburg,10,In welchem Ort ist der Job Koordinator*in / Sa...,10,0,Hamburg
1,jo100000100001,Unlimited Jobs GmbH,9,Wo ist der Job Koordinator*in / Sachbearbeiter...,10,0,Hamburg
2,jo100000100002,Hamburg,10,Wo kann man Koordinator*in / Sachbearbeiter*i...,10,0,Hamburg
3,jo100000100003,2. Koordinator*in / Sachbearbeiter*in zur Erst...,6,Wo gibt es Arbeit als Koordinator*in / Sachbea...,10,0,Hamburg
4,jo100000100004,2. Koordinator*in / Sachbearbeiter*in zur Erst...,6,Wer sucht nach einer Koordinator*in / Sachbear...,9,0,Unlimited Jobs GmbH


In [19]:
tag_predictions_dataset.to_csv('tag_predictions_dataset.csv', sep=';', index=False)

In [20]:

counter = 0
location = []
company = []
job = []
for index, row in tag_predictions_dataset.iterrows():
    if counter < 4:
        location.append(row)
    elif counter < 8:
        company.append(row)
    else:
        job.append(row)
    
    counter += 1
    if counter == 12:
        counter = 0
    

In [21]:
location_df = pd.DataFrame(location)
company_df = pd.DataFrame(company)
job_df = pd.DataFrame(job)

In [22]:
location_df


,id,predictions,tag_predictions,question,element_id,answer_start,answer
0,jo100000100000,Hamburg,10,In welchem Ort ist der Job Koordinator*in / Sa...,10,0,Hamburg
1,jo100000100001,Unlimited Jobs GmbH,9,Wo ist der Job Koordinator*in / Sachbearbeiter...,10,0,Hamburg
2,jo100000100002,Hamburg,10,Wo kann man Koordinator*in / Sachbearbeiter*i...,10,0,Hamburg
3,jo100000100003,2. Koordinator*in / Sachbearbeiter*in zur Erst...,6,Wo gibt es Arbeit als Koordinator*in / Sachbea...,10,0,Hamburg
12,jo100000100012,Bankpower GmbH,24,In welchem Ort ist der Job Mitarbeiter (gn) Re...,25,0,"Marsberg, Sauerland"
...,...,...,...,...,...,...,...
267,jo100000600267,Staplerfahrer (m/w/d) ab 14 €,38,Wo gibt es Arbeit als Staplerfahrer (m/w/d) ab...,40,0,Zwiesel
276,jo100000600276,Restaurant-Mitarbeiter:in - VZ,53,In welchem Ort ist der Job Restaurant-Mitarbei...,55,0,"Meißen, Sachsen"
277,jo100000600277,25. Restaurant-Mitarbeiter:in - VZ,51,Wo ist der Job Restaurant-Mitarbeiter:in - VZ?,55,0,"Meißen, Sachsen"
278,jo100000600278,Restaurant-Mitarbeiter:in - VZ,53,Wo kann man als Restaurant-Mitarbeiter:in - VZ...,55,0,"Meißen, Sachsen"


In [23]:
location_tag_offset = location_match(location_df)
print(location_tag_offset)

all_location_tag_offset = all_location_match(location_df)
print(all_location_tag_offset)

2.3333333333333335
2.1145833333333335


In [24]:
company_tag_offset = location_match(company_df)
print(company_tag_offset)

all_company_tag_offset = all_location_match(company_df)
print(all_company_tag_offset)

1.62
0.84375


In [25]:
job_tag_offset = location_match(job_df)
print(job_tag_offset)

all_job_tag_offset = all_location_match(job_df)
print(all_job_tag_offset)

2.466666666666667
0.3854166666666667


In [26]:
def prediction_match(df):
    counter = 0
    for index,row in df.iterrows():
        if row.predictions == row.answer:
            counter += 1
    return counter / df.shape[0]

In [27]:
location_prediction_match = prediction_match(location_df)
print(location_prediction_match)
company_prediction_match = prediction_match(company_df)
print(company_prediction_match)
job_prediction_match = prediction_match(job_df)
print(job_prediction_match)


0.09375
0.4583333333333333
0.8020833333333334
